# Modeling transmon qubit Cooper-pair box Hamiltonian in the charge basis 

Zlatko Minev, Christopher Warren, Nick Lanzillo 2021

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy = True
%matplotlib inline
%config Completer.use_jedi = False
%config InlineBackend.figure_format = 'svg'

from qiskit_metal.analyses.hamiltonian.transmon_charge_basis import Hcpb
from qiskit_metal.analyses.hamiltonian.transmon_CPB_analytic import Hcpb_analytic
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

This module models the transmon qubit in the cooper-pair charge basis, assuming wrapped junction phase variable. The Hamiltonian is given by: 

$$
\hat{H}=4E_{C}\left(\hat{n}-n_{g}\right)-E_{J}\cos\left(\hat{\phi}\right)\,,
$$

where $E_{C}$ is the charging energy, $E_{J}$ is the Josephson energy, $\hat n$ is the number of Cooper pairs transferred between charge islands, $\hat{\phi}$ is the gauge-invariant phase difference between charge islands, and $n_{g}$ is effective offset charge of the device. Expressions for the charging energy, Josephson energy and offset charge can be written as:

$$
E_{C}=\frac{e^{2}}{2C_{\Sigma}}\,,\qquad n_{g}=-\frac{C_{d}\dot{\Phi}_{s}\left(t\right)}{2e}\:,\qquad E_{J}=\frac{\phi_{0}^{2}}{L_{J}}\,,
$$

where $C_{\Sigma} = C_{J}+C_{B}+C_{g}$ (the sum of the Josephson capacitance, shunting capacitance and gate capacitance), $L_{J}$ is the inductance of the Josephson junction, and $\phi$ is the magnetic flux. 

The variables are
$$
\hat{\phi}\equiv\frac{\hat{\Phi}}{\phi_{0}},\qquad\hat{n}\equiv\frac{\hat{Q}}{2e}\,,
$$

Observe that $\hat \phi$ and $\hat n$ are both dimensiuonless, and they obey the commutation relationship:

$$
[\hat{\phi}, \hat{n}] = i
$$


The Hamiltonian can be written in the charge ($\hat n$) basis as: 

$$H=4E_\text{C}(\hat{n}-n_g)^2-\frac{1}{2}E_\text{J}\sum_n(|n\rangle\langle n+1|+\text{h.c.}),$$
Where $\hat{n} = \sum_{n=-\inf}^{\inf} |n\rangle\langle n|$

TODO:
1. describe variables. 
2. write down matrices in the charge basis.
3. add class fort analytic solutoions
4. compare


### Hcpb class

Hamiltonian-model Cooper pair box (Hcpb) class.

Used to model analytically the CPB Hamiltonian quickly
and efficiently. Solves in charge basis tridiagonal eigenvalue
problem for arbitrary Ej, Ec, ng values.

As long as nlevels remains fixed the number of charge states
considered does not change and it does not recreate the arrays,
just recomputes the properties

Returns all properties of interest for the CPB.

This model is closer to the analytic solution than the Duffing oscillator model.
Can work backwards from target qubit parameters to get the Ej, Ec or use
input Ej, Ec to find the spectrum of the Cooper Pair Box.

    @author: Christopher Warren (Chalmers University of Technology), updated by Zlatko K. Minev (IBM Quantum)
    @date: 2020, 2021




## Let's model a transmon 

### Energy levels

We can easily calculate the transition energy between states using the Hcpb class. Here, we define values of $E_{J}$, $E_{C}$ and $n_g$ and then calculate transition frequency between the first and second states as well as the anharmonicity. Units:  all energies in units of Mhz  (recall energy is ħω=hf)

In [ ]:
H = Hcpb(nlevels=15, Ej=13971.3, Ec=295.2, ng=0.001)
print(f"""
Transmon frequencies 

 ω01/2π = {H.fij(0,1): 6.0f} MHz
   α/2π = {H.anharm(): 6.0f} MHz
""")


We can compare calculated eigenvalues with the analytic solutions by using the "Hcpb_analytic" class, which calculates the transmon eigenvalues analytically using Mathieu characteristic values instead of a matrix-based approach. Let's compare the calculated values of the lowest energy at zero offset charge in both cases:  

In [ ]:
# this is using the Hcpb approach as above, solving the charge basis tridiagonal eigenvalue problem:
H_test = Hcpb(nlevels=15, Ej=13971.3, Ec=295.2, ng=0.0)

# this using the Hcpb_analytic class, which solves using the exact (analytic) solutions in terms of Mathieu characteristic values: 
H_test2 = Hcpb_analytic(Ej=13971.3, Ec=295.2, ng=0.0)

# print and compare energies 
print("E0 (calculated):", H_test.evalue_k(0))
print("E0 (analytic):", H_test2.evalue_k(0))
print("Error:", 100*(H_test2.evalue_k(0) - H_test.evalue_k(0)) / H_test2.evalue_k(0))


As we can see above, the calculated transmon eigenvalues match extremely well. 

### Wavefunctions 

We can plot the eigenstates (wavefunctions) of the transmon qubit using the commands below: 

In [ ]:
import matplotlib.pyplot as plt
for k in range (3):
    ψ, θ = H.psi_k(k)
    #plt.plot(θ, ψ.real+ψ.imag, label=f"|{k}>") # it's in either quadrature, but not both
#plt.xlabel("Junction phase θ (wrapped in the interval [-π, π])")
#plt.ylabel("Re(ψ(θ))")
#plt.legend(title="Level")

### Verifying Orthonormality of the Wavefunctions

We can verify the orthonormality of the wavefunctions. Let's first take the first two eigenstates and verify that their inner product is zero, thereby confirming orthogonality: 

In [ ]:
Psi0, theta0 = H.psi_k(0)
Psi1, theta1 = H.psi_k(1)

print(np.dot(Psi0,Psi1))

QUESTION: WHY ISN'T THIS ZERO?

Next, let's take the inner product of the first eigenstate with itself, checking that we get an output of unity:

In [ ]:
print(np.dot(Psi0, Psi0))

QUESTION: WHY ISN'T THIS ONE?

# Qutip simulation 

##### Diagonal Hamiltonian

Wrapper around Qutip to output the diagonalized
Hamiltonian truncated up to n levels of the transmon
for modeling

In [ ]:
H.h0_to_qutip(6)

##### Coupling and number operator
Wrapper around Qutip to output the number operator (charge)
for the Transmon Hamiltonian in the energy eigen-basis.
Used for computing the
coupling between other elements in the system.

In [ ]:
H.n_to_qutip(6)

Add coupling

# Experimental  

TODO: 
Describe:
Compare expeirmetnal fit and then get Ej and Ec then recoever alpha and freq and compare 
Should agree

Let's use the "params_from_spectrum" function to calculate the target Ej and Ec values for a desired transmon frequency and anharmonicity: 

In [ ]:
# 13971.3, Ec=295.2
ω, α = 5431, -341
EjEc = H.params_from_spectrum(ω, α) # set self.Ej, Cj
print(EjEc)
print("transmon frequency:", H.fij(0,1), "anharmonicity:", H.anharm())


We can also calculate the value of Ej given the value of Ec and the transmon frequency:

In [ ]:
Ec = H.params_from_freq_fixEC(ω, 295.17)
print("Ec:", Ec)

This is a little bit off from 13971.3 listed above?

%metal_heading New section on integrating sc_qubits. Do the above first before moving here.  <br><br><br><br><br><br><br>

In [ ]:
import scqubits as scq

In [ ]:
qubit = scq.Transmon(
    EJ=13.97124102543,
    EC=0.295179,
    ng=0.001,
    ncut=40,
    truncated_dim=4     # after diagonalization, we will keep 3 levels
)
evals = qubit.eigenvals(evals_count=12)
print(f"freq = {(evals[1] - evals[0])* 1000:.0f} MHz")
print(f"alpha = {((evals[2] - evals[1]) - (evals[1] - evals[0]))* 1000:.0f} MHz")

In [ ]:
qubit.plot_n_wavefunction()
qubit.plot_phi_wavefunction(which=[0,1,2], mode='real')
qubit.hamiltonian()

In [ ]:
import numpy as np
ng_list = np.linspace(-2, 2, 220)
qubit.plot_evals_vs_paramvals('ng', ng_list, evals_count=6, subtract_ground=False);

# Transmon and oscilaltor 

Interactions and interaction code - This i will send you when you are ready with the above?